In [1]:
from dask_client import submit

In [44]:
import time
import dask
from dask.distributed import Client
import os
import subprocess
from pathlib import Path
import pprint

In [23]:
%%bash
python --version 

Python 3.11.6


In [45]:
pp = pprint.PrettyPrinter()

In [57]:
def task(cmd: str):
    # log file name
    stdout = Path.cwd() / "stdout.txt"
    out_stream = open(stdout, 'wb')

    # error file name
    stderr = Path.cwd() / "stderr.txt"
    err_stream = open(stderr, 'wb')

    pid = os.getpid()
    # cmd = 'echo "Hello world! This is task 1"; pwd; uname -n'
    cmd = ["ls", "-l"]
    # subprocess.run(cmd, shell=True, stdout=out_stream, stderr=err_stream,)
    out = subprocess.run(cmd, shell=True, capture_output=True, cwd=Path.cwd(), text=True)
    return out

In [11]:
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")
# create a client
client = Client(scheduler_file=scheduler_file)

In [12]:
client

<Client: 'tcp://10.249.43.208:8786' processes=128 threads=32768, memory=59.57 TiB>

In [58]:
# submit a task
num_tasks = 10_000
futures = []
for i in range(num_tasks):
    futures.append(client.submit(task, "Hello World"))

# wait for the task to finish
futures = client.gather(futures)

In [60]:
print(len(futures))
pp.pprint(futures[1].stdout)

10000
('01-dask-cpu-module-example.ipynb\n'
 '02-dask-cpu-conda-example.ipynb\n'
 '03-dask-gpu-container-example.ipynb\n'
 'dask-calculate-pi.py\n'
 'kernel.json\n'
 'launch-dask-conda-cpu.sh\n'
 'launch-dask-module-cpu.sh\n'
 'launch-dask-shifter-gpu.sh\n'
 'README.md\n'
 'stderr.txt\n'
 'stdout.txt\n'
 'submit-dask-batch.slurm\n')


2024-01-03 17:02:12,895 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [40]:
pprint.print(futures[0])

AttributeError: 'function' object has no attribute 'print'

In [ ]:
pp =